<a href="https://colab.research.google.com/github/pelinbalci/LLM_Notebooks/blob/main/Basic_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%env PIP_DISABLE_PIP_VERSION_CHECK True
%env PIP_ROOT_USER_ACTION ignore

%pip install -q -U torch==2.0.1 bitsandbytes==0.39.1 --root-user-action=ignore
%pip install -q -U datasets py7zr einops tensorboardX --root-user-action=ignore
%pip install -q -U git+https://github.com/huggingface/transformers.git@850cf4af0ce281d2c3e7ebfc12e0bc24a9c40714 --root-user-action=ignore
%pip install -q -U git+https://github.com/huggingface/peft.git@e2b8e3260d3eeb736edf21a2424e89fe3ecf429d --root-user-action=ignore
%pip install -q -U git+https://github.com/huggingface/accelerate.git@b76409ba05e6fa7dfc59d50eee1734672126fdba --root-user-action=ignore
!pip install transformers
!pip install torch

env: PIP_DISABLE_PIP_VERSION_CHECK=True
env: PIP_ROOT_USER_ACTION=ignore
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
# Basic Inference with T5

from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig


# Use AutoModelForseq2Seq for model and AutoTokenizer for tokenizer.
model_name = 'google/flan-t5-base'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)


# Configurations
generation_config = GenerationConfig(max_new_tokens=50,
                                     do_sample=True,
                                     temperature=0.7)

# Sample data
huggingface_dataset_name = "knkarthick/dialogsum"
dataset = load_dataset(huggingface_dataset_name)
example_index = [40]  # select an example
dialogue = dataset['test'][example_index]['dialogue']  # get the dialogue
summary = dataset['test'][example_index]['summary'] # get the human summary

# Encode input
inputs_encoded = tokenizer(dialogue, return_tensors='pt')
# Model Output
model_output = model.generate(inputs_encoded["input_ids"], generation_config=generation_config)[0]

# Decode the output
zero_output = tokenizer.decode(model_output, skip_special_tokens=True)

print("Dialogue:")
print(dialogue)
print("-------------------------")
print("summary:")
print(summary)
print("-------------------------")
print("inputs_encoded:")
print(inputs_encoded)
print("-------------------------")
print("model_output:")
print(model_output)
print("-------------------------")
print("zero_output:")
print(zero_output)



Dialogue:
["#Person1#: What time is it, Tom?\n#Person2#: Just a minute. It's ten to nine by my watch.\n#Person1#: Is it? I had no idea it was so late. I must be off now.\n#Person2#: What's the hurry?\n#Person1#: I must catch the nine-thirty train.\n#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there."]
-------------------------
summary:
['#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.']
-------------------------
inputs_encoded:
{'input_ids': tensor([[ 1713,   345, 13515,   536,  4663,    10,   363,    97,    19,    34,
             6,  3059,    58,  1713,   345, 13515,   357,  4663,    10,  1142,
             3,     9,  1962,     5,    94,    31,     7,     3,   324,    12,
          4169,    57,    82,  1605,     5,  1713,   345, 13515,   536,  4663,
            10,    27,     7,    34,    58,    27,   141,   150,   800,    34,
            47,    78,  1480,     5,    27,   

In [ ]:
# Basic inference 2:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "EleutherAI/pythia-70m"
base_model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

device_count = torch.cuda.device_count()
if device_count > 0:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

base_model.to(device)

test_text = "How does gender equality benefit society?"
max_input_tokens = 1000
max_output_tokens=100
# Tokenize
input_ids = tokenizer.encode(
      test_text,
      return_tensors="pt",
      truncation=True,
      max_length=max_input_tokens
)

# Generate
generated_tokens_with_prompt = base_model.generate(input_ids=input_ids.to(device), max_length=max_output_tokens)

# Decode
generated_text_with_prompt = tokenizer.batch_decode(generated_tokens_with_prompt, skip_special_tokens=True)

# Strip the prompt
generated_text_answer = generated_text_with_prompt[0][len(test_text):]


print("Question input (test):", test_text)
print("Model's answer: ")
print(generated_text_answer)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Question input (test): How does gender equality benefit society?
Model's answer: 


A:

The only way to get rid of this is to use the "gender equality" option.

A:

The only way to get rid of this is to use the "gender equality" option.

A:

You can use the "gender equality" option.

A:

You can use the "gender equality" option.

A:

You can use the "gender equality
